In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.3.0-dev20200522'

In [2]:
PATH = 'data'

os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [3]:
#train = pd.read_csv(f'{PATH}/train.csv')
train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')

train.shape, val.shape

((2220, 8), (10932, 8))

In [4]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [5]:
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE

def decode(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img, label

def augment(img, label):
    # augmentation
    #img = tf.image.flip_left_right(img)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img, label

def load_ds(df):
    options = tf.data.Options()
    options.experimental_deterministic = False
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224/train/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.with_options(options)
    ds = ds.map(decode, num_parallel_calls=AUTO)
    ds = ds.map(augment, num_parallel_calls=AUTO)
    ds = ds.cache()
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTO)
    return ds

In [6]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [7]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable = False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [9]:
filepath = './checkpoints/checkpoint'
cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = filepath,
    monitor="val_auc",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="max"
)

In [10]:
model.fit(
    train_ds, 
    epochs=10, 
    validation_data=val_ds, 
    validation_steps=10,
    callbacks=[cb]
)

Epoch 1/10
35/35 [==============================] - ETA: 0s - loss: 0.2114 - auc: 0.4682
Epoch 00001: val_auc improved from -inf to 0.42938, saving model to ./checkpoints\checkpoint
35/35 [==============================] - 12s 348ms/step - loss: 0.2114 - auc: 0.4682 - val_loss: 0.2147 - val_auc: 0.4294
Epoch 2/10
35/35 [==============================] - ETA: 0s - loss: 0.1114 - auc: 0.4761
Epoch 00002: val_auc did not improve from 0.42938
35/35 [==============================] - 10s 294ms/step - loss: 0.1114 - auc: 0.4761 - val_loss: 0.0913 - val_auc: 0.3804
Epoch 3/10
35/35 [==============================] - ETA: 0s - loss: 0.0940 - auc: 0.5853
Epoch 00003: val_auc improved from 0.42938 to 0.61178, saving model to ./checkpoints\checkpoint
35/35 [==============================] - 11s 308ms/step - loss: 0.0940 - auc: 0.5853 - val_loss: 0.1707 - val_auc: 0.6118
Epoch 4/10
35/35 [==============================] - ETA: 0s - loss: 0.0820 - auc: 0.7275
Epoch 00004: val_auc improved from 0.61

In [11]:
model.load_weights(filepath)
model.save("model.h5")

In [12]:
model.evaluate(val_ds)

171/171 [==============================] - 38s 224ms/step - loss: 0.0790 - auc: 0.7835


[0.07904893904924393, 0.7834699749946594]

In [22]:
m = tf.keras.metrics.AUC()
aucs = []
for imgs, labels in val_ds:
    imgs_lr = tf.image.flip_left_right(imgs)    
    imgs_ud = tf.image.flip_up_down(imgs)
    preds = (model.predict(imgs) + model.predict(imgs_lr) + model.predict(imgs_ud)) / 3
    _ = m.update_state(labels, preds)
    aucs.append(m.result().numpy())

In [24]:
import numpy as np

np.mean(aucs)

0.78355503

In [26]:
test = pd.read_csv(f'{PATH}/test.csv')

test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [27]:
def decode_test(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img

def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [28]:
test_ds = load_test_ds(test)

In [30]:
preds = []
for imgs in test_ds:
    imgs_lr = tf.image.flip_left_right(imgs)    
    imgs_ud = tf.image.flip_up_down(imgs)
    _preds = (model.predict(imgs) + model.predict(imgs_lr) + model.predict(imgs_ud)) / 3
    preds += _preds.ravel().tolist()

In [31]:
preds

[0.007331361994147301,
 0.005941618233919144,
 0.012626026757061481,
 0.0026187915354967117,
 0.00838464591652155,
 0.013834868557751179,
 0.012857361696660519,
 0.41657090187072754,
 0.08851868659257889,
 0.011794954538345337,
 0.013195042498409748,
 0.0007183433626778424,
 0.049807388335466385,
 0.002018091967329383,
 0.04832886531949043,
 0.007651734631508589,
 0.003820915473625064,
 0.002840500557795167,
 0.021782152354717255,
 0.12551088631153107,
 0.019332218915224075,
 0.0043656048364937305,
 0.05111469328403473,
 0.015146116726100445,
 0.05755317211151123,
 0.013213424943387508,
 0.0014402768574655056,
 0.006858881562948227,
 0.0022502734791487455,
 0.006640955805778503,
 0.017257345840334892,
 0.0053505562245845795,
 0.15367169678211212,
 0.10828373581171036,
 0.04836838319897652,
 0.0028221197426319122,
 0.020707381889224052,
 0.05226681008934975,
 0.003334806300699711,
 0.0066464669071137905,
 0.004613333847373724,
 0.06816787272691727,
 0.0629257783293724,
 0.00146055256482

In [32]:
submission = pd.DataFrame({'image_name': test['image_name'].values, 'target': preds})

submission

,image_name,target
0,ISIC_0052060,0.007331
1,ISIC_0052349,0.005942
2,ISIC_0058510,0.012626
3,ISIC_0073313,0.002619
4,ISIC_0073502,0.008385
...,...,...
10977,ISIC_9992485,0.024654
10978,ISIC_9996992,0.038456
10979,ISIC_9997917,0.028613
10980,ISIC_9998234,0.002834


In [33]:
submission.to_csv('submission.csv', index=False)